# Análise de Variabilidade Semântica (Colab)

Notebook para executar o script `analise_variabilidade.py` passo a passo no Google Colab.

Escolhe a célula de instalação, depois a de configuração (path para o ficheiro no Drive) e corre todas.

In [ ]:
# Instalar dependências (executar apenas se necessário)
!pip install -q spacy openpyxl beautifulsoup4 lxml nltk
!python -m spacy download pt_core_news_lg


In [ ]:
import pandas as pd
import re
import spacy
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import wordnet as wn
from datetime import datetime
import os

nltk.download('wordnet')
nltk.download('omw-1.4')

nlp = spacy.load('pt_core_news_lg')
print('Configuração carregada.')

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# CONFIGURAÇÃO: caminho do ficheiro XML no teu Drive e tipo de construção
FILE_PATH = '/content/drive/MyDrive/Constructions_concordances/SVO_7000.xml'  # ALTERA AQUI
tipo_construcao = 'svo'  # 'svo', 'n_adj', 'adj_n'
print('FILE_PATH =', FILE_PATH)
print('tipo_construcao =', tipo_construcao)


In [ ]:
def limpar_kwic(texto_kwic):
    return re.sub(r"/[a-zA-Z]+", "", texto_kwic)

def extrair_svo(frase):
    doc = nlp(frase)
    sujeito = verbo = objeto = None
    for token in doc:
        if token.dep_ == 'ROOT' and token.pos_ == 'VERB':
            verbo = token.lemma_
        elif token.dep_ == 'nsubj':
            sujeito = token.text
        elif token.dep_ in {'obj','dobj','obl','attr'}:
            objeto = token.lemma_
    if verbo and (sujeito or objeto):
        return {'frase_limpa': frase, 'sujeito': sujeito or '', 'verbo': verbo, 'objeto': objeto or ''}
    return None

def extrair_n_adj(frase):
    doc = nlp(frase)
    for token in doc:
        if token.pos_ == 'ADJ' and token.head.pos_ == 'NOUN':
            return {'frase_limpa': frase, 'nome': token.head.lemma_, 'adjetivo': token.lemma_}
    return None

def extrair_adj_n(frase):
    doc = nlp(frase)
    for token in doc:
        if token.pos_ == 'ADJ' and token.head.pos_ == 'NOUN' and token.i < token.head.i:
            return {'frase_limpa': frase, 'adjetivo': token.lemma_, 'nome': token.head.lemma_}
    for i in range(len(doc)-1):
        if doc[i].pos_ == 'ADJ' and doc[i+1].pos_ == 'NOUN':
            return {'frase_limpa': frase, 'adjetivo': doc[i].lemma_, 'nome': doc[i+1].lemma_}
    return None


In [ ]:
# Leitura do XML
if not os.path.exists(FILE_PATH):
    raise FileNotFoundError(f'Arquivo não encontrado: {FILE_PATH}')

with open(FILE_PATH, 'r', encoding='utf-8') as f:
    xml = f.read()

soup = BeautifulSoup(xml, 'xml')
kwics = soup.find_all('kwic')
kwic_pairs = [(limpar_kwic(k.text.strip()), k.text.strip()) for k in kwics]

print(f'KWICs lidos: {len(kwic_pairs)}')


In [ ]:
# Construção do DataFrame
dados = []
for frase_limpa, frase_original in kwic_pairs:
    if tipo_construcao == 'svo':
        extraido = extrair_svo(frase_limpa)
    elif tipo_construcao == 'n_adj':
        extraido = extrair_n_adj(frase_limpa)
    elif tipo_construcao == 'adj_n':
        extraido = extrair_adj_n(frase_limpa)
    else:
        extraido = None
    if extraido:
        extraido['frase_original'] = frase_original
        extraido['frase_limpa'] = frase_limpa
        dados.append(extraido)

df = pd.DataFrame(dados)
print(f'Frases extraídas: {len(df)}')


In [ ]:
# Mapeamento domínios (igual ao script)
mapeamento_dominios = {
    'noun.person': 'pessoa',
    'noun.artifact': 'objeto',
    'noun.act': 'evento',
    'noun.event': 'evento',
    'noun.group': 'organização',
    'noun.location': 'lugar',
    'noun.communication': 'comunicação',
    'noun.state': 'estado',
    'noun.cognition': 'conhecimento',
    'noun.quantity': 'quantidade',
    'noun.attribute': 'característica',
    'noun.time': 'tempo',
    'noun.animal': 'animal',
    'noun.body': 'corpo',
    'noun.food': 'comida',
    'noun.substance': 'matéria',
    'noun.object': 'objeto',
    'noun.feeling': 'emoção',
    'noun.phenomenon': 'fenómeno',
}

def obter_dominios_note(book_word, lang='por'):
    if not book_word:
        return 'desconhecido', 'desconhecido'
    synsets = wn.synsets(book_word, lang=lang)
    if not synsets:
        return 'desconhecido', 'desconhecido'
    primeiro = synsets[0]
    subdominio = primeiro.lexname()
    dominio_mapeado = mapeamento_dominios.get(subdominio, 'outro')
    return dominio_mapeado, subdominio

# Atribui domínios no DataFrame (conforme tipo de construção)
if tipo_construcao == 'svo':
    df['dominio'], df['subdominio'] = zip(*df['objeto'].apply(lambda x: obter_dominios_note(x)))
    df['dominio_sujeito'], df['subdominio_sujeito'] = zip(*df['sujeito'].apply(lambda x: obter_dominios_note(x)))
    df['construcao'] = df['verbo'].apply(lambda v: f"{v} X")
elif tipo_construcao == 'n_adj':
    df['dominio'], df['subdominio'] = zip(*df['nome'].apply(lambda x: obter_dominios_note(x)))
    df['construcao'] = df.apply(lambda r: f"{r['nome']} + {r['adjetivo']}", axis=1)
elif tipo_construcao == 'adj_n':
    df['dominio'], df['subdominio'] = zip(*df['nome'].apply(lambda x: obter_dominios_note(x)))
    df['construcao'] = df.apply(lambda r: f"{r['adjetivo']} {r['nome']}", axis=1)

df.head()


In [ ]:
# Cálculo de variabilidade (SVO com verbo->obj e verbo->suj)
if tipo_construcao == 'svo':
    agrupados_verbo_obj = df.groupby('verbo')['dominio'].apply(list)
    df_var_verbo_obj = agrupados_verbo_obj.apply(lambda x: len(set(x))).reset_index(name='variabilidade_verbo_obj')

    agrupados_verbo_suj = df.groupby('verbo')['dominio_sujeito'].apply(list)
    df_var_verbo_suj = agrupados_verbo_suj.apply(lambda x: len(set(x))).reset_index(name='variabilidade_verbo_suj')

    display(df_var_verbo_obj.sort_values(by='variabilidade_verbo_obj', ascending=False).head(10))
    display(df_var_verbo_suj.sort_values(by='variabilidade_verbo_suj', ascending=False).head(10))
elif tipo_construcao == 'n_adj':
    df['dominio_adjetivo'], df['subdominio_adjetivo'] = zip(*df['adjetivo'].apply(lambda x: obter_dominios_note(x)))
    agrupados_nome = df.groupby('nome')['dominio_adjetivo'].apply(list)
    df_var_nome = agrupados_nome.apply(lambda x: len(set(x))).reset_index(name='variabilidade_nome')
    display(df_var_nome.sort_values(by='variabilidade_nome', ascending=False).head(10))
elif tipo_construcao == 'adj_n':
    agrupados = df.groupby('adjetivo')['dominio'].apply(list)
    df_var = agrupados.apply(lambda x: len(set(x))).reset_index(name='variabilidade_semantica')
    display(df_var.sort_values(by='variabilidade_semantica', ascending=False).head(10))


In [ ]:
# Exportar para Excel no Drive
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_path = f"/content/drive/MyDrive/Constructions_concordances/output_{tipo_construcao}_{timestamp}.xlsx"

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name='Construcoes')
    if tipo_construcao == 'svo':
        df_var_verbo_obj.to_excel(writer, index=False, sheet_name='Variabilidade_verbo_objeto')
        df_var_verbo_suj.to_excel(writer, index=False, sheet_name='Variabilidade_verbo_sujeito')
    elif tipo_construcao == 'n_adj':
        df_var_nome.to_excel(writer, index=False, sheet_name='Variabilidade_nome')
    elif tipo_construcao == 'adj_n':
        df_var.to_excel(writer, index=False, sheet_name='Variabilidade')

print('✅ Exportado para:', output_path)
